In [22]:
%matplotlib inline
from ipyleaflet import *
from ipywidgets import HTML, Layout, IntSlider, RadioButtons, Dropdown, Button, Output
from IPython.display import display
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Variable setup
x, y = 0, 0
dummy_num = 0


In [3]:
# Map setup
defaultLayout=Layout(width='1000px', height='800px')
center=(35,20)
m = Map(center=center,
       #basemap=basemap_to_tiles(basemaps.NASAGIBS.ModisTerraTrueColorCR, "2022-01-01"),
       basemap=basemaps.Gaode.Satellite,
        zoom=4,
       layout=defaultLayout)
def handle_click(**kwargs):
    if kwargs.get('type') == 'click':
        #m.add_layer(Marker(location=kwargs.get('coordinates')))
        #print("You clicked on: ", kwargs.get('coordinates'))
        
        global x
        global y
        global dummy_num
        
        x, y = kwargs.get('coordinates')
        dummy_num += 1
        message1 = HTML()
        message1.value = f"Coordinates: ({np.round(x,2)}, {np.round(y,2)}) <br> Aquaculture name: XY <br> Number: {dummy_num}"
        popup = Popup(
            location=(x, y),
            child=message1,
            close_button=True,
            auto_close=True,
            close_on_escape_key=True,
            max_width = 400,
            name = "Location info"
        )
        m.add_layer(popup)

m.on_interaction(handle_click)

In [4]:
# Display map
m

Map(center=[35, 20], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_tex…

In [27]:
# Print results
radios = RadioButtons(options=["species 1", "species 2", "species 3"],
    description="Choose a species!",
    disabled=False
)

display(radios)

btn = Button(description="Simulate!")
out = Output()


def on_button_clicked(obj):
    plt.scatter(np.linspace(0,10), np.linspace(0,10))
    plt.show()

btn.on_click(on_button_clicked)

display(btn)

RadioButtons(description='Choose a species!', options=('species 1', 'species 2', 'species 3'), value='species …

Button(description='Simulate!', style=ButtonStyle())